In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import re

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from pprint import pprint

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
%matplotlib inline


/gpfs/helios/home/volodymyr1/src/bird_clef_2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib.util>:247: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.


`speechbrain` was not imported
`LEAF` was not imported


/gpfs/helios/home/volodymyr1/src/bird_clef_2025/.venv/lib/python3.11/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


# Export Models

In [3]:
!ls -lt ../logdirs/ | head -20

total 23
drwxr-xr-x  9 volodymyr1 users 4096 Apr 24 17:30 eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromV2Y2025Last_PseudoF1PT05MT01P04I2
drwxr-xr-x  8 volodymyr1 users 4096 Apr 24 16:45 tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_AdamW1e4_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromPrebs1
drwxr-xr-x 10 volodymyr1 users 4096 Apr 24 15:34 eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromPreca4_PseudoF1PT05MT01P04
drwxr-xr-x 10 volodymyr1 users 4096 Apr 23 13:11 tf_efficientnetv2_s_in21k_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_

In [ ]:
# bird2id_source = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp.json")
# bird2id_target = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json")

# id2bird_source = {v:k for k,v in bird2id_source.items()}
# id2bird_target = {v:k for k,v in bird2id_target.items()}

# REARRANGE_INDICES = np.array([
#     bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
# ]).astype(int)

# STRICT_LOAD = False

# def prune_checkpoint_rule(inp_chkp):
#     inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
#     inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
#     inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
#     inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

#     return inp_chkp

In [ ]:
# len(bird2id_source)

In [4]:
MODEL_CLASS = WaveCNNAttenClasifier
TRAIN_PERIOD = 5

In [6]:
EXP_NAME = "eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromV2Y2025Last_PseudoF1PT05MT01P04"

conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

from glob import glob

import torch

from code_base.augmentations.transforms import BackgroundNoise, OneOf
from code_base.callbacks import ROC_AUC_Score
from code_base.datasets import WaveAllFileDataset, WaveDataset
from code_base.forwards import MultilabelClsForwardLongShort
from code_base.models import RandomFiltering, WaveCNNAttenClasifier
from code_base.train_functions.train_lightning import lightning_training

B_S = 64
TRAIN_PERIOD = 5.0
N_EPOCHS = 50
ROOT_PATH = "data/train_audio"
LATE_NORMALIZE = True
MAXIMIZE_METRIC = True
MAIN_METRIC = "valid_roc_auc"
PATH_TO_JSON_MAPPING = (
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/bird2int_2025.json"
)
PRECOMPUTE = False
REPLACE_PATHES = ("train_audio", "train_features")
DEBUG = False
N_CORES = 10

CONFIG = {
    "seed": 1243,
    "df_path": "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025_hdf5.csv",

In [7]:
MODELS = [
    {
        "model_config": dict(
            backbone="eca_nfnet_l0",
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 206,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
                "output_type": "clipwise_pred_long",
            },
            exportable=True,
            fixed_amplitude_to_db=True
        ),
        "exp_name": "eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromPreca4_PseudoF1PT05MT01P04",
        "fold": [0, 1, 2, 3, 4],
        "chkp_name":"last.ckpt",
        "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
        "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
        "delete_prefix": "model.",
        "n_swa_models": 1,
        "model_output_key": None,
        # "prune_checkpoint_func": prune_checkpoint_rule
    },
    {
        "model_config": dict(
            backbone="eca_nfnet_l0",
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 206,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
                "output_type": "clipwise_pred_long",
            },
            exportable=True,
            fixed_amplitude_to_db=True
        ),
        "exp_name": "eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromV2Y2025Last_PseudoF1PT05MT01P04",
        "fold": [0, 1, 2, 3, 4],
        "chkp_name":"last.ckpt",
        "swa_checkpoint_regex": r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
        "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
        "delete_prefix": "model.",
        "n_swa_models": 1,
        "model_output_key": None,
        # "prune_checkpoint_func": prune_checkpoint_rule
    },
]

In [8]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [9]:
model = []
for config in MODELS:
    model.extend([create_model_and_upload_chkp(
        model_class=MODEL_CLASS,
        model_config=config['model_config'],
        model_device="cpu",
        model_chkp_root=f"../logdirs/{config['exp_name']}/fold_{m_i}/checkpoints",
        model_chkp_basename=config["chkp_name"] if config["swa_checkpoint_regex"] is None else None,
        model_chkp_regex=config.get("swa_checkpoint_regex"),
        swa_sort_rule=config.get("swa_sort_rule"),
        n_swa_to_take=config.get("n_swa_models", 3),
        delete_prefix=config.get("delete_prefix"),
        prune_checkpoint_func=config.get("prune_checkpoint_func")
    ) for m_i in config["fold"]])

SWA checkpoints
[{'name': 'epoch=12-step=6032-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'},
 {'name': 'epoch=34-step=16240-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'},
 {'name': 'epoch=35-step=16704-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'}]
SWA sorted checkpoints
[{'name': 'epoch=12-step=6032-valid_roc_auc=0.983.ckpt',
  'valid_roc_auc': '0.983'}]
vanilla model
Loading ../logdirs/eca_nfnet_l0_Exp_noamp_64bs_5sec_mixupP05_RandomFiltering_SqrtBalancing_Radamlr1e3_CosBatchLR1e6_Epoch50_BackGroundSoundScapeORESC50P05_SpecAugV1_FocalBCELoss_LSF1005_5Folds_ScoredPrevCompsAndXCsnipet28032025_FromPreca4_PseudoF1PT05MT01P04/fold_0/checkpoints/epoch=12-step=6032-valid_roc_auc=0.983.ckpt
STFT kernels created, time used = 0.6985 seconds
Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
SWA checkpoints
[{'name': 'epoch=41-step=16758-valid_roc_auc=0.984.ckpt',
  'valid_roc_auc': '0.984'},
 {'name': 'epoch=39-step=15960-valid_roc_auc=0.984.ckpt',


In [ ]:
exportable_ensem = ONNXEnsemble(
    model_class=MODEL_CLASS,
    configs=list(chain(*[
        [deepcopy(config['model_config']) for _ in config["fold"]] for config in MODELS
    ])),
    # weights=[2.0] * 5 + [1.0] * 5
)
assert len(exportable_ensem.models) == len(model)
for model_id in range(len(model)):
    exportable_ensem.models[model_id].load_state_dict(model[model_id].state_dict())
    exportable_ensem.models[model_id].eval()
exportable_ensem.eval()
convert_to_onnx(
    model_to_convert=exportable_ensem,
    sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
    base_path=f"../logdirs/double_eca_nfnet_l0_AllFPseudoI1/onnx_ensem_5first_folds",
    use_fp16=True,
    use_openvino=True,
    opset_version=12
)

STFT kernels created, time used = 0.4301 seconds
STFT kernels created, time used = 0.5264 seconds
STFT kernels created, time used = 0.5504 seconds
STFT kernels created, time used = 0.6238 seconds
STFT kernels created, time used = 0.5319 seconds
STFT kernels created, time used = 0.5449 seconds
STFT kernels created, time used = 0.5106 seconds
STFT kernels created, time used = 0.4456 seconds
STFT kernels created, time used = 0.5579 seconds
STFT kernels created, time used = 0.5386 seconds


/gpfs/helios/home/volodymyr1/src/bird_clef_2025/.venv/lib/python3.11/site-packages/nnAudio/features/stft.py:283: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.num_samples < self.pad_amount:
/gpfs/helios/home/volodymyr1/src/bird_clef_2025/.venv/lib/python3.11/site-packages/torch/onnx/_internal/jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/gpfs/helios/home/volodymyr1/src/bird_clef_2025/.venv/lib/python3.11/site-packages/torch/onnx/symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMod

In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
!ls -lt ../logdirs/ | head

# Check

In [ ]:
os.listdir(f"../logdirs/{EXP_NAME}/")

In [ ]:
core = ov.Core()
model = core.read_model(model=f"../logdirs/{EXP_NAME}/onnx_ensem_openvino_fp16/model_simpl.xml")
compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
sample = np.random.randn(5, 32000*5).astype(np.float16)

In [ ]:
compiled_model.output(0)

In [ ]:
%%time
output = compiled_model([sample])[compiled_model.output(0)]

In [ ]:
output.shape

In [ ]:
output.dtype

In [ ]:
sample = sample.astype(np.float16)

In [ ]:
onnx_model = ort.InferenceSession(
    f"../logdirs/{EXP_NAME}/onnx_ensem_fp16/model_simpl.onnx"
)

In [ ]:
wave = torch.from_numpy(np.random.randn(32000*5)).half()

In [ ]:
wave.numpy()

In [ ]:
%%time
out = onnx_model.run(
    None,
    {"input": sample}
)

In [ ]:
out[0].shape

In [ ]:
if CONFIG["run_test"]:
    inference_class_onnx = BirdsInference(
        device="cpu",
        verbose_tqdm=True,
        use_sigmoid=CONFIG["use_sigmoid"],
        model_output_key=CONFIG["model_output_key"],
    )
    onnx_model = ort.InferenceSession(
        f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
    )
    test_preds_onnx, test_preds_long_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
        nn_models=onnx_model,
        data_loader=loader_test,
        is_onnx_model=True
    )
    test_pred_df_onnx = compose_submission_dataframe(
        probs=test_preds_onnx,
        dfidxs=test_dfidx_onnx,
        end_seconds=test_end_onnx,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )